In [1]:
import abacus_cosmos.Halos as ach
from astropy.table import Table
import matplotlib.pyplot as plt
import astropy.cosmology as apc
import os
import numpy as np
import glob
from astropy import constants as const
from collections import Counter
%matplotlib inline

In [2]:
dtype=[('BoxID','i8'), ('hubble', 'f8'), ('omega_de', 'f8'),
      ('omega_m', 'f8'), ('n_s', 'f8'), ('sigma_8', 'f8'), ('w_0', 'f8')]
cosmo_data = np.loadtxt("../abacus/box_cosmo_params.dat", dtype=dtype)

In [39]:
def get_sphere(box_id=0, RSD=False):
    main_path = "/Users/forero/data/AbacusCosmos/"
    halo_path = "AbacusCosmos_720box_{:02d}_FoF_halos_z0.100".format(box_id)
    halo_data = Table(ach.read_halos_FoF(os.path.join(main_path,halo_path)))

    vcirc_max_limit = 300.0
    r_limit = 300
    #hubble = cosmo_data['hubble'][box_id]/100
       
    halo_data['r_comov'] = np.sqrt(np.sum(halo_data['pos']**2, axis=1))
    halo_data['v_radial'] = np.sum(halo_data['pos']*halo_data['vel'], axis=1)/halo_data['r_comov']

    if RSD:
        halo_data['r'] = halo_data['r_comov'] + halo_data['v_radial']/100
    else:
        halo_data['r'] = halo_data['r_comov'].copy()
        
    #halo_data['r'] = halo_data['r']/hubble

    ii = (halo_data['r']<r_limit) & (halo_data['vcirc_max']>vcirc_max_limit) & (halo_data['r']>0)
    print(box_id, np.count_nonzero(ii))
    
    if RSD:
        halo_data['pos'][:,0] = halo_data['pos'][:,0] + (halo_data['pos'][:,0]/halo_data['r_comov'])*(halo_data['v_radial']/100)
        halo_data['pos'][:,1] = halo_data['pos'][:,1] + (halo_data['pos'][:,1]/halo_data['r_comov'])*(halo_data['v_radial']/100)
        halo_data['pos'][:,2] = halo_data['pos'][:,2] + (halo_data['pos'][:,2]/halo_data['r_comov'])*(halo_data['v_radial']/100)

        
    sphere_data = halo_data['pos'][ii]
    n_points = len(sphere_data)
    
    random_pos = sphere_data.copy()
    phi  = np.random.random(n_points)* np.pi * 2.0
    theta = np.arccos(2.0*(np.random.random(n_points)-0.5))
    r = np.sqrt(np.sum(sphere_data**2, axis=1))
    random_pos[:,0] = r * np.cos(phi) * np.sin(theta)
    random_pos[:,1] = r * np.sin(phi) * np.sin(theta)
    random_pos[:,2] = r * np.cos(theta)
    
    if RSD:
        output_filename = "../abacus/sphere_{:02d}_RSD.dat".format(box_id)
        random_filename = "../abacus/sphere_random_{:02d}_RSD.dat".format(box_id)
    else:
        output_filename = "../abacus/sphere_{:02d}.dat".format(box_id)
        random_filename = "../abacus/sphere_random_{:02d}.dat".format(box_id)

    np.savetxt(output_filename, sphere_data)
    np.savetxt(random_filename, random_pos)
    del halo_data
    del sphere_data
    del random_pos

In [41]:
for i in range(40):
    get_sphere(box_id=i, RSD=True)
    get_sphere(box_id=i, RSD=False)

0 75389
0 75022
1 83193
1 82843
2 62789
2 62508
3 94757
3 94294
4 65626
4 65330
5 77779
5 77399
6 64143
6 63905
7 68915
7 68587
8 66025
8 65736
9 69460
9 69120
10 87843
10 87403
11 64639
11 64351
12 82971
12 82623
13 89034
13 88652
14 73644
14 73259
15 66394
15 66087
16 67500
16 67185
17 83161
17 82799
18 82220
18 81806
19 71172
19 70833
20 82533
20 82171
21 84854
21 84453
22 70021
22 69655
23 98110
23 97665
24 70606
24 70262
25 97834
25 97423
26 74991
26 74645
27 80371
27 80012
28 81388
28 81013
29 84311
29 83882
30 78354
30 78010
31 81345
31 80967
32 68652
32 68381
33 78168
33 77851
34 82920
34 82507
35 79908
35 79498
36 88049
36 87671
37 65730
37 65422
38 59773
38 59502
39 68210
39 67889
